# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [48]:
import glob
import numpy as np
import scipy as sp
import scipy.signal as sp_signal
import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [40]:
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

In [41]:
def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

In [42]:
def FourierTransform(signal, fs, factor=6):
    """
    Computes the frequencies and the magnitudes of the Fourier Transform
    over a signal
    
    Args:
        signal: (np.array) the signal to be transformed
        fs: (number) sampling rate
    
    Returns:
        frequencies and magnitudes of the signal
    """
    window_length = factor * len(signal)
    
    frequencies = np.fft.rfftfreq(window_length, 1 / fs)
    fft = np.abs(np.fft.rfft(signal, window_length))
    
    return frequencies, fft

In [51]:
def ButterworthFilter(signal, pass_band=(40/60, 240/60), fs=125):
    """
    Butterworth filter algorithm.
    
    Returns:
        Bandpass filtered signal
    """          
    b, a = sp_signal.butter(2, pass_band, btype='bandpass', fs = fs)
    return sp_signal.filtfilt(b, a, signal)

In [52]:
def filter_signal(signal):
    signal[signal <= 40/60] = 0
    signal[signal >= 240/60] = 0
    return signal

In [53]:
def Featurize(ppg, accx, accy, accz, fs):
    """
    Featurization of the signal
    
    Args:
        ppg: (np.array) photoplethysmography signal
        accx: (np.array) x-channel of the accelerometer signal
        accy: (np.array) y-channel of the accelerometer signal
        accz: (np.array) z-channel of the accelerometer signal
        fs: (number) sampling rate of the accelerometer signal
    
    Returns:
        n-tuple of PPG and ACC features
    """
    
    # Compute accelerator value with its three channels
    acc = np.sqrt(np.sum(np.array([accx, accy, accz]) ** 2, axis=0))
    
    # Fourier Transform
    ppg_freqs, ppg_fft = FourierTransform(ppg, fs)
    acc_freqs, acc_fft = FourierTransform(acc, fs)

    # Filter ffts
    ppg_fft = filter_signal(ppg_fft)
    acc_fft = filter_signal(acc_fft)
    
    # Features
    ppg_mean_freq = np.mean(ppg_freqs)
    ppg_mean_fft = np.mean(ppg_fft)
    ppg_max_freq = ppg_freqs[np.argmax(ppg_fft)]
    acc_mean_freq = np.mean(acc_freqs)
    acc_mean_fft = np.mean(acc_fft)
    acc_max_freq = acc_freqs[np.argmax(acc_fft)]
    
    return (ppg_mean_freq, 
            ppg_mean_fft, 
            ppg_max_freq,
            acc_mean_freq,
            acc_mean_fft,
            acc_max_freq,
           )

In [54]:
class DataProcessor:
    def __init__(self, data_fls, ref_fls, fs=100, window_length=8, window_shift=2):
        # Data
        self.data_fls = data_fls
        self.ref_fls = ref_fls
        # Hyperparameters
        self.fs = fs
        self.window_length = window_length
        self.window_shift = window_shift
        
        if len(self.data_fls) == 0: self.data_fls = ''
    
    def __len__(self):
        return len(self.data_fls)

    def __getitem__(self, idx):
        # Lists to store features, labels and signals
        features, labels, signals = [], [], []
        
        # Extract data
        ppg, accx, accy, accz = LoadTroikaDataFile(self.data_fls[idx])
        bpms = sp.io.loadmat(self.ref_fls[idx])['BPM0'][:, 0]

        # Process data
        total_windows = min(len(ppg), len(bpms))
        left_indexes = np.arange(total_windows, dtype=int) * self.fs * self.window_shift
        right_indexes = left_indexes + self.fs * self.window_length
        
        # Iterate over windows
        for i, (left, right) in enumerate(zip(left_indexes, right_indexes)):
            # Extract portions
            ppg_win = ButterworthFilter(ppg[left:right])
            accx_win = ButterworthFilter(accx[left:right])
            accy_win = ButterworthFilter(accy[left:right])
            accz_win = ButterworthFilter(accz[left:right])
            # Save features, labels and signals
            features.append(Featurize(ppg_win, accx_win, accy_win, accz_win, self.fs))
            labels.append(bpms[i])
            signals.append([ppg_win, accx_win, accy_win, accz_win])

        return features, labels, signals, (ppg, accx, accy, accz)

class Estimator():
    """
    Top-level class for the estimator model
    
    Generates a RandomForestRegressor model and provides functions to interact with it.
    
    """
    def __init__(self, processor, n_estimators=100, max_depth=10):
        # Parameters
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        
        # Model
        self.model = RandomForestRegressor()
        
        # Hyperparameters
        self.processor = processor
    
    def train(self):
        # Generate features
        features, labels, _ = self.generate_features()
        # Split dataset
        X_train, _, y_train, _ = train_test_split(features, labels, test_size=0.2)

        # Train model
        self.model.fit(X_train, y_train)
    
    def predict(self, X):
        return self.model.predict(X)
            
    def generate_features(self):
        features, labels, signals = [], [], []

        for i in range(len(self.processor)):
            _features, _labels, _signals, _ = self.processor[i]
            features.extend(_features)
            labels.extend(_labels)
            signals.extend(_signals)

        return np.array(features), np.array(labels), np.array(signals)

In [55]:
# replace the code below with your pulse rate algorithm.
import numpy as np
import scipy as sp
import scipy.io

def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Runs the pulse rate algorithm on the given data files, trains a model and evaluates its
    performance on the data computing the MAE and the confidence metric.
    """
    data_fls, ref_fls = LoadTroikaDataset()
    # Generate classes
    processor = DataProcessor(data_fls, ref_fls)
    model = Estimator(processor=processor)
    # Train model
    model.train()

    # Predict
    errors, confidences = [], []
    for i in range(len(processor)):
        # Predict
        features, labels, signals, _ = processor[i]
        predictions = model.predict(features)

        _errors, _confidences = [], []
        for i in range(predictions.shape[0]):
            ppg = ButterworthFilter(signals[i][0])

            freqs, fft = FourierTransform(ppg, processor.fs, 2)
            fft[fft <= 40/60] = 0
            fft[fft >= 240/60] = 0

            pred_fs = predictions[i] / 55
            pred_fs_win = (freqs >= pred_fs - 0.5) & (freqs <= pred_fs + 0.5)
            confs = np.sum(fft[pred_fs_win]) / (np.sum(fft) + 1e-6)
            _confidences.append(confs)
            _errors.append(np.abs(predictions[i] - labels[i]))
        
        errors.append(_errors)
        confidences.append(_confidences)
    
    errors = np.hstack(errors)
    confidences = np.hstack(confidences)
    
    return errors, confidences